In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [11]:
import yaml

# 1. Lire le fichier YAML depuis OneLake
yaml_path = "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Files/gold.yaml"
yaml_content = spark.read.text(yaml_path).collect()
yaml_str = "\n".join([row.value for row in yaml_content])
config = yaml.safe_load(yaml_str)

# 2. Extraire tous les source_path
sources_to_process = []
for table in config["tables_gold"]:
    if "bronze" in table and "source_path" in table["bronze"]:
        sources_to_process.append(table["bronze"]["source_path"])

# 3. Créer une chaîne de paramètres
sources_arg = ",".join(sources_to_process)

# 4. Appeler le notebook `notebook_load_data` avec le paramètre
result = mssparkutils.notebook.run(
    "notebook_load_data",     # ⚠️ mets bien le nom réel du notebook dans Fabric
    300,
    { "sources": sources_arg }
)

print("✅ Notebook load_data exécuté avec succès")


StatementMeta(, bfaeb8ac-086d-460c-af55-76bf94ff09ea, 13, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o5470.throwExceptionIfHave.
: com.microsoft.spark.notebook.msutils.NotebookExecutionException: name 'df_valid' is not defined
---------------------------------------------------------------------------NameError                                 Traceback (most recent call last)Cell In[16], line 4
      1 notebook_name = "notebook_load_data"
      2 mode = "SCD1"
----> 4 insert_bronze_to_silver(final_config, mode, notebook_name)
Cell In[11], line 76, in insert_bronze_to_silver(config, mode, notebook_name)
     43 log_data = {
     44     "job_id": job_id,  
     45     "orchestration_id": 0,  
   (...)
     63     "day": int(job_start.day)
     64 }
     66 log_job_execution(log_data)  
     68 flow_data = {
     69     "job_id": job_id,
     70     "flow_id": flow_id,
     71     "file_path":f"abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_bronze.Lakehouse/Tables/{table['name']}",
     72     "flow_start":job_start,
     73     "flo

In [12]:
import yaml
from pyspark.sql import SparkSession
from datetime import datetime
import getpass
import uuid
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, ShortType

# 📌 **1. Lire le fichier YAML depuis OneLake**
yaml_path = "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Files/gold.yaml"

# Charger le contenu du fichier YAML
yaml_content = spark.read.text(yaml_path).collect()
yaml_str = "\n".join([row.value for row in yaml_content])
config = yaml.safe_load(yaml_str)

# 🛠 Debug 1 : Vérifier le contenu du YAML
print("🔍 Structure du YAML chargé en dict Python :")
print(config)

# 📌 **2. Extraire dynamiquement les sources à traiter**
sources_to_process = []

if "tables_gold" in config and config["tables_gold"]:  # Vérification
    for table in config["tables_gold"]:
        if "sources" in table:
            for source in table["sources"]:
                if "source_path" in source:
                    sources_to_process.append(source["source_path"])
else:
    print("❌ ERREUR : `tables_gold` est absent ou vide dans le fichier YAML !")
    sources_to_process = []  # Liste vide pour éviter une erreur

# Supprimer les doublons
sources_to_process = list(set(sources_to_process))

print(f"✅ Sources extraites du YAML : {sources_to_process}")

# 📍 **3. Sauvegarde des sources dans OneLake**
sources_arg = ",".join(sources_to_process)

spark.createDataFrame([(sources_arg,)], ["sources"])\
    .write.mode("overwrite").option("header", "true")\
    .csv("Files/tmp/params_sources")


# 📍 **4. Informations d'orchestration**
orchestration_start = datetime.now()
user_name = getpass.getuser()
workspace_name = "fabric_workspace"
environment = "dev"
data_product_name = "load_employee_data"
domain_name = "HR"
entity_name = "employee"
orchestration_name = "data_pipeline_execution"
orchestration_id = int(uuid.uuid4().int % (10**10))  
suplementary_run_id = str(uuid.uuid4())

# ▶️ **5. Lancer le notebook cible**
try:
    result = mssparkutils.notebook.run("notebook_load_data", 300, {})
    orchestration_status = "SUCCESS"
    orchestration_message = result
except Exception as e:
    orchestration_status = "FAILED"
    orchestration_message = str(e)

orchestration_end = datetime.now()

# 📝 **6. Créer un dictionnaire contenant toutes les infos de log**
log_data = {
    "orchestration_id": orchestration_id,
    "suplementary_run_id": suplementary_run_id,
    "orchestration_name": orchestration_name,
    "data_product_name": data_product_name,
    "domain_name": domain_name,
    "entity_name": entity_name,
    "environment": environment,
    "workspace_name": workspace_name,
    "user_name": user_name,
    "orchestration_start": orchestration_start,
    "orchestration_end": orchestration_end,
    "orchestration_status": orchestration_status,
    "orchestration_message": orchestration_message,
    "year": orchestration_start.year,
    "month": orchestration_start.month,
    "day": orchestration_start.day
}



#  **8. Appel de la fonction `log_orchestration`**
log_orchestration(log_data)

#  **Message final**
print(f"🎯 Processus terminé avec statut : {orchestration_status}")


StatementMeta(, 3c0067cc-e31f-4eb3-9d33-c8bae970e399, 14, Finished, Available, Finished)

🔍 Structure du YAML chargé en dict Python :
{'tables_gold': [{'name': 'gold_employee', 'target': 'lakehouse_gold.gold_employee', 'sources': [{'name': 'employee', 'source_path': 'Files/employee/inbound/'}, {'name': 'education_level', 'source_path': 'Files/education_level/inbound/'}, {'name': 'performance_rating', 'source_path': 'Files/performance/inbound/'}]}, {'name': 'gold_performance_rating', 'target': 'lakehouse_gold.gold_performance_rating', 'sources': [{'name': 'performance_rating', 'source_path': 'Files/performance/inbound/'}, {'name': 'employee', 'source_path': 'Files/employee/inbound/'}, {'name': 'satisfied_level', 'source_path': 'Files/satisfaction/inbound/'}, {'name': 'rating_level', 'source_path': 'Files/rating/inbound/'}]}]}
 Sources extraites du YAML : ['Files/employee/inbound/', 'Files/satisfaction/inbound/', 'Files/rating/inbound/', 'Files/education_level/inbound/', 'Files/performance/inbound/']


📌 Log orchestration ajouté avec statut : SUCCESS


In [2]:
from datetime import datetime
import uuid
import getpass
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, ShortType

StatementMeta(, 2e9b26d7-8293-4210-8588-25ac04691581, 12, Finished, Available, Finished)

In [3]:
%run "notebook_function"

StatementMeta(, 2e9b26d7-8293-4210-8588-25ac04691581, 22, Finished, Available, Finished)

In [3]:
import yaml

# 📌 Lire le YAML
yaml_path = "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Files/gold.yaml"
yaml_content = spark.read.text(yaml_path).collect()
yaml_str = "\n".join([row.value for row in yaml_content])
config = yaml.safe_load(yaml_str)

#extraction de source
sources_to_process = extract_sources_from_yaml(config)

# 💾 Écrire les sources dans un fichier temporaire
sources_arg = ",".join(sources_to_process)
spark.createDataFrame([(sources_arg,)], ["sources"])\
    .write.mode("overwrite").option("header", "true")\
    .csv("Files/tmp/params_sources")

# lancer notebook load data
try:
    result = mssparkutils.notebook.run("notebook_load_data", 300, {})
    print(" Notebook load_data exécuté avec succès")
except Exception as e:
    print(f" Échec de l'exécution du notebook load_data : {str(e)}")


StatementMeta(, 14091fc1-fb3c-4d3a-be4e-2237d13ccde5, 5, Finished, Available, Finished)

📌 Log orchestration ajouté avec statut : SUCCESS


✅ Notebook load_data exécuté avec succès


In [4]:
import yaml

# 📌 Lire le YAML
yaml_path = "abfss://test@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Files/gold.yaml"
yaml_content = spark.read.text(yaml_path).collect()
yaml_str = "\n".join([row.value for row in yaml_content])
config = yaml.safe_load(yaml_str)

# 🔍 Extraction des sources via ta fonction
sources_to_process = extract_sources_from_yaml(config)

# ✅ Convertir en string séparée par virgule
sources_arg = ",".join(sources_to_process)

# ▶️ Appel du notebook avec paramètre
try:
    result = mssparkutils.notebook.run(
        "notebook_load_data",
        300,
        { "sources": sources_arg }
    )
    print("✅ Notebook load_data exécuté avec succès")
except Exception as e:
    print(f"❌ Échec de l'exécution du notebook load_data : {str(e)}")


StatementMeta(, 2e9b26d7-8293-4210-8588-25ac04691581, 23, Finished, Available, Finished)

📌 Log orchestration ajouté avec statut : SUCCESS


❌ Échec de l'exécution du notebook load_data : An error occurred while calling o6885.throwExceptionIfHave.
: com.microsoft.spark.notebook.msutils.NotebookExecutionException: name 'sources_arg' is not defined
---------------------------------------------------------------------------NameError                                 Traceback (most recent call last)Cell In[4], line 4
      1 # Lire le fichier contenant les paramètres
      2 #df = spark.read.option("header", "true").csv("Files/tmp/params_sources")
      3 #sources_arg = df.collect()[0]["sources"]
----> 4 sources_to_process = sources_arg.split(",")
      6 print(f"📥 Sources à traiter : {sources_to_process}")
      8 # 👉 PAS de return ici
NameError: name 'sources_arg' is not definedYou can check driver log or snapshot for detailed error info! See how to check logs: https://go.microsoft.com/fwlink/?linkid=2157243 .
	at com.microsoft.spark.notebook.workflow.JobSessionClient.runCell(JobSessionClient.scala:237)
	at com.microsoft.spark

In [4]:
df = spark.sql("SELECT * FROM monitoring.orchestration LIMIT 1000")
display(df)

StatementMeta(, 14091fc1-fb3c-4d3a-be4e-2237d13ccde5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 46dab997-7371-45ef-b47b-eae715622b11)

In [5]:
df = spark.sql("SELECT * FROM monitoring.flow LIMIT 1000")
display(df)

StatementMeta(, 14091fc1-fb3c-4d3a-be4e-2237d13ccde5, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fa82cd20-7074-458c-bbe7-abd4828fee9b)